In [1]:
import numpy as np
import pandas as pd
from itertools import product

# from pyomo.environ import *
import pyomo.environ as e

# Define data
attrs = ['den', 'bnz', 'roz', 'moz']

sources_data = {
    "s1": [49.2, 6097.56, {'den': 0.82, 'bnz':3, 'roz':99.2,'moz':90.5}],
    "s2": [62.0, 16129, {'den': 0.62, 'bnz':0, 'roz':87.9,'moz':83.5}],
    "s3": [300.0, 500, {'den': 0.75, 'bnz':0, 'roz':114,'moz':98.7}]
}

targets_data = {
    "t1": [190, 500, {'den': 0.74, 'roz':95, 'moz':85, 'bnz':0}, {'den': 0.79, 'roz':0, 'moz':0, 'bnz':0}],
    "t2": [230, 500, {'den': 0.74, 'roz':96, 'moz':88, 'bnz':0}, {'den': 0.79, 'roz':0, 'moz':0, 'bnz':0.9}],
    "t3": [150, 500, {'den': 0.74, 'roz':91, 'moz':0 , 'bnz':0}, {'den': 0.79, 'roz':0, 'moz':0, 'bnz':0}]
}

pools_data = {
    "p1": 1250,
    "p2": 1750
}

In [2]:
sources = list(sources_data.keys())
targets = list(targets_data.keys())
pools = list(pools_data.keys())

cost = {s: sources_data[s][0] for s in sources}
supply = {s: sources_data[s][1] for s in sources}
content = {s: sources_data[s][2] for s in sources}

price = {t: targets_data[t][0] for t in targets}
demand = {t: targets_data[t][1] for t in targets}
min_tol = {t: targets_data[t][2] for t in targets}
max_tol = {t: targets_data[t][3] for t in targets}

cap = {p: pools_data[p] for p in pools}

# p_pooling
p_pooling = e.ConcreteModel()

# e.Sets
p_pooling.sources = e.Set(initialize=sources)
p_pooling.targets = e.Set(initialize=targets)
p_pooling.pools = e.Set(initialize=pools)
p_pooling.attrs = e.Set(initialize=attrs)

# e.Parameters
p_pooling.cost = e.Param(p_pooling.sources, initialize=cost)
p_pooling.supply = e.Param(p_pooling.sources, initialize=supply)
p_pooling.content = e.Param(p_pooling.sources, within=e.Any, initialize=content)

p_pooling.price = e.Param(p_pooling.targets, initialize=price)
p_pooling.demand = e.Param(p_pooling.targets, initialize=demand)
p_pooling.min_tol = e.Param(p_pooling.targets, within=e.Any, initialize=min_tol)
p_pooling.max_tol = e.Param(p_pooling.targets, within=e.Any, initialize=max_tol)

p_pooling.cap = e.Param(p_pooling.pools, initialize=cap)

# Decision e.Variables
p_pooling.ik = e.Var(p_pooling.sources, p_pooling.targets, domain=e.NonNegativeReals)
p_pooling.ij = e.Var(p_pooling.sources, p_pooling.pools, domain=e.NonNegativeReals)
p_pooling.jk = e.Var(p_pooling.pools, p_pooling.targets, domain=e.NonNegativeReals)
p_pooling.prop = e.Var(p_pooling.pools, p_pooling.attrs, domain=e.NonNegativeReals)

In [3]:
# e.Constraints
# The fluids transit integrally from sources to targets
def flow_conservation_rule(model, j):
    return sum(model.ij[i, j] for i in model.sources) == sum(model.jk[j, k] for k in model.targets)
p_pooling.flow_conservation_con = e.Constraint(p_pooling.pools, rule=flow_conservation_rule)

# The outgoing flow from sources does not exceed supply amounts from each source
def source_capacity_rule(model, i):
    return sum(model.ij[i, j] for j in model.pools) + sum(model.ik[i, k] for k in model.targets) <= model.supply[i]
p_pooling.source_capacity_con = e.Constraint(p_pooling.sources, rule=source_capacity_rule)

# The outgoing flow from each pool does not exceed pool inventory capacity
def pool_capacity_rule(model, j):
    return sum(model.jk[j, k] for k in model.targets) <= model.cap[j]
p_pooling.pool_capacity_con = e.Constraint(p_pooling.pools, rule=pool_capacity_rule)

# The incoming flow in targets is no less than the demand at each target
def target_demand_rule(model, k):
    return sum(model.ik[i, k] for i in model.sources) + sum(model.jk[j, k] for j in model.pools) >= model.demand[k]
p_pooling.target_demand_con = e.Constraint(p_pooling.targets, rule=target_demand_rule)

# The outgoing concentration from e.Any pool of each attribute is the weighted average of incoming concentrations from sources
def pool_concentration_rule(model, j, attr):
    return sum(model.content[i][attr] * model.ij[i, j] for i in model.sources) == model.prop[j, attr] * sum(model.jk[j, k] for k in model.targets)
p_pooling.pool_concentration_con = e.ConstraintList()

for j in pools:
    for attr in attrs:
        # p_pooling.pool_concentration_con.add(p_pooling.prop[j, attr] == 0) # "== 0" ? prev. function not used ?
        p_pooling.pool_concentration_con.add(pool_concentration_rule(p_pooling, j, attr))

# The incoming resulting concentration at each target is no less than the minimum required concentration at such target
def target_mintolerance_rule(model, k, attr):
    return sum(model.content[i][attr] * model.ik[i, k] for i in model.sources) + sum(model.prop[j, attr] * model.jk[j, k] for j in model.pools) >= \
               model.min_tol[k][attr] * (sum(model.ik[i, k] for i in model.sources) + sum(model.jk[j, k] for j in model.pools))

def target_maxtolerance_rule(model, k, attr):
    return sum(model.content[i][attr] * model.ik[i, k] for i in model.sources) + sum(model.prop[j, attr] * model.jk[j, k] for j in model.pools) >= \
               model.min_tol[k][attr] * (sum(model.ik[i, k] for i in model.sources) + sum(model.jk[j, k] for j in model.pools))

p_pooling.target_min_tolerance_con = e.ConstraintList()
p_pooling.target_max_tolerance_con = e.ConstraintList()
for k in targets:
    for attr in min_tol[k].keys():
        # p_pooling.target_min_tolerance_con.add(sum(p_pooling.content[i][attr] * p_pooling.ik[i, k] for i in p_pooling.sources) + sum(p_pooling.prop[j, attr] * p_pooling.jk[j, k] for j in p_pooling.pools) >= p_pooling.min_tol[k][attr] * (sum(p_pooling.ik[i, k] for i in p_pooling.sources) + sum(p_pooling.jk[j, k] for j in p_pooling.pools)))
        # p_pooling.target_max_tolerance_con.add(sum(p_pooling.content[i][attr] * p_pooling.ik[i, k] for i in p_pooling.sources) + sum(p_pooling.prop[j, attr] * p_pooling.jk[j, k] for j in p_pooling.pools) <= p_pooling.max_tol[k][attr] * (sum(p_pooling.ik[i, k] for i in p_pooling.sources) + sum(p_pooling.jk[j, k] for j in p_pooling.pools)))
        p_pooling.target_min_tolerance_con.add(target_mintolerance_rule(p_pooling, k, attr))
        p_pooling.target_min_tolerance_con.add(target_maxtolerance_rule(p_pooling, k, attr))

In [4]:
# Objective function
def total_profit_rule(model):
    return(sum(model.price[k] * 
                    (sum(model.ik[i, k] for i in model.sources) + sum(model.jk[j, k] for j in model.pools)) for k in model.targets) - 
           sum(model.cost[i] * 
                    (sum(model.ij[i, j] for j in model.pools) + sum(model.ik[i, k] for k in model.targets)) for i in model.sources)
          )

p_pooling.total_profit_obj = e.Objective(rule=total_profit_rule, sense=e.maximize)

In [5]:
# Solve the model
solver = e.SolverFactory('gurobi')
results = solver.solve(p_pooling, tee= True)

# Print results
if results.solver.status == e.SolverStatus.ok and results.solver.termination_condition == e.TerminationCondition.optimal:
    print("Optimal solution found!")
    print("Objective Value:", e.value(p_pooling.total_profit_obj))
else:
    print("Solver terminated with status:", results.solver.status)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-19
Read LP format model from file C:\Users\adame\AppData\Local\Temp\tmpad_4b24k.pyomo.lp
Reading time = 0.00 seconds
x1: 10 rows, 29 columns, 48 nonzeros
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-11300H @ 3.10GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10 rows, 29 columns and 48 nonzeros
Model fingerprint: 0x11b45683
Model has 32 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e-02, 1e+02]
  Objective range  [5e+01, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+02, 2e+04]

Continuous model is non-convex -- solving as a MIP

Presolve time: 0.00s
Presolved: 162 rows, 62 columns, 452 nonzeros
Presolved model has 32 biline

Questions:
- Timesteps for Blending problem ?
- 